In [1]:
## use the summary json file ot  answer questions based on the data 
## do embedding

In [2]:
from utils import get_openai_key


key = get_openai_key()

/Users/anubhavtomar/2023_projects/chatgpt/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [1]:
## open file 
from pathlib import Path
from dirs import *
import json

file_name = 'morepen_summary_32.json'
def load_summary(file_name)->dict:
    file_path = Path(SUMMARY_DIR,file_name)
    with open(file_path, 'r') as fr:
        return json.load(fr)
    
def load_summary_embedded(file_name)->dict:
    file_path = Path(EMBEDDING_DIR,file_name)
    with open(file_path, 'r') as fr:
        return json.load(fr)
    
def save_summary(doc, file_name):
     file_path =  Path(SUMMARY_DIR,file_name)
     with open(file_path, 'w') as fw:
        return json.dump(doc,fw)


In [ ]:

doc = load_summary(file_name)
type(doc)

In [5]:
from utils import get_openai_client
from utils_openai import get_embedding


/Users/anubhavtomar/2023_projects/chatgpt/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [11]:
# for k,v in doc.items():
#     doc[k]['embedding'] = get_embedding(client,doc[k]['chunk_text']) 

In [6]:
## save the embedded doc
# save_summary(doc,f'embedded_{file_name}')

In [10]:
doc = load_summary_embedded(file_name)
client = get_openai_client()

In [11]:

question = "What was the overall performance of the company in the last quarter?"
question_embedding = get_embedding(client,question)


In [12]:

import numpy as np


def cosine_similarity(a, b):
    a = np.array(a)
    b = np.array(b)
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Function to find top N most relevant chunks based on the question
def find_top_chunks(doc, question_embedding, top_n=3):
    similarities = []
    for key, value in doc.items():
        chunk_embedding = value['embedding']
        similarity = cosine_similarity(question_embedding, chunk_embedding)
        similarities.append((key, similarity))
    
    # Sort chunks by similarity in descending order
    similarities.sort(key=lambda x: x[1], reverse=True)
    
    # Get the top N chunks
    top_chunks = [doc[key] for key, _ in similarities[:top_n]]
    return top_chunks


In [13]:
top_chunks = find_top_chunks(doc, question_embedding)

In [14]:
len(top_chunks)

3

In [15]:
summaries = [c['summary'] for c in top_chunks]
texts = [c['chunk_text'] for c in top_chunks]

In [16]:
texts

["r is usually slow in this business. Formulations have grown 2%. Seventy-one percent of the business comes from the medicine side, and 29% of the business comes from the OTC side. On the distribution side, we have more than 5,000 distributors, 5,100 distributors. We are present in 3 lakh outlets, 3.28 lakh outlets taken together. And all over the country, we are present. 580 people are there in all the states for the selling. The GenMix are distributed separately to a separate company. It was a single distributor. But otherwise, we have got service centers. We have got distributors all over the country. I'm not getting into detail of plans. Financial numbers have already been circulated. So as I shared that gross revenue during the quarter – I'm talking about the quarter now. During the quarter, the gross revenues have gone up 16.5%. EBITDA has gone up 182%, from 18 crores to 52 crores EBITDA. EBITDA level has gone from 5% to 12.31%. So it's more than double as a percentage because th

In [17]:
def create_prompt(top_chunks, question):
    prompt = "You are an expert analyst. Answer the following question based on the provided sections of a document.\n\n"
    
    for chunk in top_chunks:
        section = chunk['section']
        # summary = chunk['summary']
        texts = chunk['chunk_text']
        prompt += f"{section}: {texts}\n\n"
    
    prompt += f"Question: {question}\n"
    return prompt

In [18]:
prompt = create_prompt(top_chunks,question)

In [19]:
from constants import DEFAULT_TEMPERATURE, DEFAULT_TOP_P


system_content =  "You are a helpful assistant."
client = get_openai_client() 
def get_answer_from_gpt(client,
                        prompt,
                    system_content = system_content,
                    model = "gpt-3.5-turbo",
                    max_tokens = 150):
    


    response = client.chat.completions.create(
                                                model=model,
                                                messages=[
                                                    {"role": "system", "content":system_content},
                                                    {"role": "user","content":prompt}],
                                                temperature=DEFAULT_TEMPERATURE,
                                                max_tokens=max_tokens,
                                                top_p=DEFAULT_TOP_P
                                            )
    answer = response.choices[0].message.content
    return  answer

In [20]:
get_answer_from_gpt(client=client,prompt=prompt,)

'In the last quarter, the company showed significant growth in its financial performance. The gross revenue increased by 16.5%, EBITDA rose by 182%, and profit before tax almost tripled. Earnings per share increased by 144% to Rs. 1.88 for a Rs. 2 share. Return on equity doubled to 11%, and return on capital employed nearly doubled to 15%. The company also reported a 279% increase in profit before tax and a 259% increase in profit after tax. Overall, the company demonstrated strong financial performance in the last quarter.'

In [47]:
from constants import DEFAULT_MODEL


def classify_question(client,question):
    classification_prompt = f"I will provide you with a prompt given by a user to an AI agent trained on a document. Use your intelligence to understand the intent of the user and classify the question as either : SPECIFIC or SUMMARY. return either of the two words in response. Heres the question as given by user: \n\nQuestion: {question}\n"

    
    response = client.chat.completions.create(
                                                model=DEFAULT_MODEL,
                                                messages=[
                                                    {"role": "system", "content":system_content},
                                                    {"role": "user","content":classification_prompt}],
                                                temperature=DEFAULT_TEMPERATURE,
                                                max_tokens=300,
                                                top_p=DEFAULT_TOP_P
                                            )
    classification = response.choices[0].message.content.strip().lower()
    return classification


In [42]:
def create_summary_prompt(doc):
    print("creating summary prompt")
    prompt = "You are an expert analyst. Summarize the entire document based on the provided sections.\n\n"
    count = 0
    for key, chunk in doc.items():
        if count ==20:
            break
        else:
            section = chunk['section']
            summary = chunk['summary']
            prompt += f"{section}: {summary}\n\n"
            count +=1
    return prompt

In [36]:
count_tokens(create_summary_prompt(doc))

3215

In [48]:
from utils_openai import count_tokens


def answer_question(doc, question):
    client = get_openai_client()
    classification = classify_question(client, question)
    print(classification)
    if classification == "summary":
        
        prompt = create_summary_prompt(doc)
        print(count_tokens(prompt))
    else:pass 
        # question_embedding = get_embedding(client,question)
        # top_chunks = find_top_chunks(doc, question_embedding,top_n=3)
        # summaries = [c['summary'] for c in top_chunks]
        # texts = [c['chunk_text'] for c in top_chunks]
        # print(summaries)
        # print(texts)
        # prompt = create_prompt(top_chunks, question)
        # print(count_tokens(prompt))
    answer = get_answer_from_gpt(client,prompt)
    return answer

In [26]:
question = "what are the key promises given by the management for the coming quarters? what metrics should be increasing to show growth?"
question = "summarise the whole concall in details including various sections of the call and outlining the important financials etc. around 1000 words "

In [49]:
question

'summarise the whole concall in details including various sections of the call and outlining the important financials etc. around 1000 words '

In [50]:
answer = answer_question(doc, question)
answer

summary
creating summary prompt
3215


"Based on the provided sections, the document summarizes Morepen Laboratories Limited's performance in Q4 FY24 earnings conference call. The company operates in two main segments - medical devices and APIs, with a focus on home medical devices and leading products in glucose monitors and blood pressure monitors. The management highlights significant revenue growth, becoming debt-free, and achieving strong financial performance. They emphasize the company's expansion plans into international markets, capacity increase, and focus on technology advancements. The management discusses market leadership in APIs and long-term customer relationships, with a focus on maintaining high-quality standards. The document also covers the company's financial performance, operational details, and future revenue growth projections. Analyst Q&A sessions address concerns about margins, distribution channels, competition, and growth potential"